In [1]:
from numpy.random import choice, normal, uniform, binomial
from numpy import sum, mean, zeros, array, NaN
import pandas as pd
from math import comb

In [2]:
pd.options.mode.chained_assignment = None

# Simulating Data

We will simulate two types of users (enthusiasts and normal), whith proportion `prob_enthusiast`. Enthusiast have a higher impact on the outcome `enthusiast_effect` and also a higher acceptance rate `inc_enth_cons` of doing the recommended items. So the type of users is an **unobserved confounder**. 

Each user does `sessions_n` sessions, and at each session, there are only `k` items recommended. There is an underlying recommender system (the recommender system by default) that always recommends items 0:`k` for normal users, while always recommends the last `k` items to enthusiasts. Every session, with probability `exploration_prob` the items are recommended uniformly at random. 

The outcome is calculated as follows. Each treatment `i` has an additve causal impact of `i` (to make it easy the index of the item is the same causal impact). The outcome is the addition of each item consumed, plus some noise `delta`. As already said, enthusiast have an extra bonus of `enthusiast_effect` in the outcome.

In [3]:
treatments_n = 10
patients_n = 10000
sessions_n = 2
k = 3
delta = 0.1
exploration_prob = 0.8

prob_enthusiast = 0.2
enthusiast_effect = 10
base_consumption = 0.2
inc_enth_cons = 0.8

In [4]:
ids = []
sessions = []
patient_types = []
treatments = []
recommendations = []
consumptions = []
originals = []
explorations = []

treats_ids = list(range(treatments_n))
for id in range(patients_n):
    patient_type = choice([0, 1], size=1, p=[1-prob_enthusiast, prob_enthusiast])[0]
    for session in range(sessions_n):
        exploration = binomial(1, exploration_prob, size=1)[0]
        original = None
        if patient_type == 1:
            original = treats_ids[-k:]
        else:
            original = treats_ids[:k]
        if exploration == 1:
            recommended = choice(treats_ids, size=k, replace=False)
        else:
            recommended = original
            
        recommended = [int(treat in recommended) for treat in treats_ids]     
        original = [int(treat in original) for treat in treats_ids]     
        
        consumption = []
        for treat in range(treatments_n):
            score_assign = base_consumption + inc_enth_cons*patient_type
            item_cons = int(uniform(size=1)[0] <= score_assign)
            item_cons *= recommended[treat]
            consumption.append(item_cons)
        
        ids += [id]*treatments_n
        sessions += [session]*treatments_n
        patient_types += [patient_type]*treatments_n
        treatments += treats_ids
        recommendations += recommended
        consumptions += consumption
        explorations += [exploration]*treatments_n
        originals += original
    
df = pd.DataFrame({
    'id': ids,
    'session': sessions, 
    'patient_type': patient_types,
    'treatment': treatments, 
    'original': originals,
    'recommended': recommendations,
    'consumed': consumptions,
    'exploration': explorations
})

outcomes = df.groupby('id').apply(lambda x: 
    (sum(x['treatment']*x['consumed']) + 
    mean(x['patient_type'])*enthusiast_effect + 
    normal(size=1, scale=delta))[0]
).reset_index()
outcomes.rename(columns={0:'outcome'}, inplace = True)
df = df.merge(outcomes, on='id')

In [5]:
df.head(n=treatments_n*sessions_n)

,id,session,patient_type,treatment,original,recommended,consumed,exploration,outcome
0,0,0,0,0,1,1,0,0,10.94773
1,0,0,0,1,1,1,0,0,10.94773
2,0,0,0,2,1,1,1,0,10.94773
3,0,0,0,3,0,0,0,0,10.94773
4,0,0,0,4,0,0,0,0,10.94773
5,0,0,0,5,0,0,0,0,10.94773
6,0,0,0,6,0,0,0,0,10.94773
7,0,0,0,7,0,0,0,0,10.94773
8,0,0,0,8,0,0,0,0,10.94773
9,0,0,0,9,0,0,0,0,10.94773


In [6]:
recs = df.pivot(index=['id', 'session', 'exploration'], columns='treatment', values='recommended').reset_index().merge(outcomes, on='id')
consumes = df.pivot(index=['id', 'session', 'exploration'], columns='treatment', values='consumed').reset_index().merge(outcomes, on='id')
originals = df.pivot(index=['id', 'session', 'exploration'], columns='treatment', values='original').reset_index().merge(outcomes, on='id')

In [7]:
recs.head(sessions_n)

,id,session,exploration,0,1,2,3,4,5,6,7,8,9,outcome
0,0,0,0,1,1,1,0,0,0,0,0,0,0,10.94773
1,0,1,1,1,0,0,0,0,0,0,1,0,1,10.94773


In [8]:
consumes.head(sessions_n)

,id,session,exploration,0,1,2,3,4,5,6,7,8,9,outcome
0,0,0,0,0,0,1,0,0,0,0,0,0,0,10.94773
1,0,1,1,1,0,0,0,0,0,0,0,0,1,10.94773


In [9]:
originals.head(sessions_n)

,id,session,exploration,0,1,2,3,4,5,6,7,8,9,outcome
0,0,0,0,1,1,1,0,0,0,0,0,0,0,10.94773
1,0,1,1,1,1,1,0,0,0,0,0,0,0,10.94773


# Basic Stats

In [10]:
df[df.recommended==1].groupby('patient_type').agg({'consumed': [mean]})

,consumed
,mean
patient_type,
0,0.198667
1,1.000000


In [11]:
df.groupby('patient_type').agg({'outcome': [mean]})

,outcome
,mean
patient_type,
0,4.584892
1,41.427724


# Real Impact of Recommendations

This are the difference of causal impact between pairs of items (overlook the zeros, the matrix is actually symetric)

In [12]:
expected_diff_treats = zeros((treatments_n, treatments_n))

expected_compliers = base_consumption*(1-prob_enthusiast) + (base_consumption + inc_enth_cons)*prob_enthusiast

for treat_1 in range(treatments_n - 1):
    for treat_2 in range(treat_1 + 1, treatments_n):
        expected_diff_treats[treat_1, treat_2] = (treat_1 - treat_2)*expected_compliers

pd.DataFrame(expected_diff_treats.round(2))

,0,1,2,3,4,5,6,7,8,9
0,0.0,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88,-3.24
1,0.0,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88
2,0.0,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52
3,0.0,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16
4,0.0,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80
5,0.0,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Direct Estimation Recommendations

This is the difference between recommendations obtained by trivial computations. You can see they are biased.

In [13]:
diff_treats_0 = zeros((treatments_n, treatments_n))

for treat_1 in range(treatments_n - 1):
    for treat_2 in range(treat_1 + 1, treatments_n):
        try: 
            inds_1_0 = recs.loc[:, treat_1] == 1
            inds_2_0 = recs.loc[:, treat_2] == 1
            res_1 = recs[inds_1_0].loc[:, 'outcome'].mean() - recs[inds_2_0].loc[:, 'outcome'].mean()
        except:
            res_1 = NaN
        diff_treats_0[treat_1, treat_2] = res_1
            
pd.DataFrame(diff_treats_0.round(2))

,0,1,2,3,4,5,6,7,8,9
0,0.0,-0.17,-0.39,-4.14,-4.58,-4.25,-4.89,-10.27,-10.58,-10.96
1,0.0,0.00,-0.22,-3.97,-4.41,-4.09,-4.72,-10.10,-10.41,-10.80
2,0.0,0.00,0.00,-3.75,-4.19,-3.87,-4.50,-9.88,-10.19,-10.58
3,0.0,0.00,0.00,0.00,-0.44,-0.12,-0.75,-6.13,-6.44,-6.83
4,0.0,0.00,0.00,0.00,0.00,0.32,-0.31,-5.69,-6.00,-6.39
5,0.0,0.00,0.00,0.00,0.00,0.00,-0.64,-6.01,-6.33,-6.71
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-5.38,-5.69,-6.08
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.31,-0.70
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.38
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In comparison with the theoretical results

In [14]:
pd.DataFrame(expected_diff_treats.round(2))

,0,1,2,3,4,5,6,7,8,9
0,0.0,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88,-3.24
1,0.0,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88
2,0.0,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52
3,0.0,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16
4,0.0,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80
5,0.0,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Our Method

In [15]:
diff_treats = zeros((treatments_n, treatments_n))

N = treatments_n
q = exploration_prob/comb(N-2, k-1)

for treat_1 in range(treatments_n - 1):
    for treat_2 in range(treat_1 + 1, treatments_n):
        other_treatments = [t for t in treats_ids if t not in [treat_1, treat_2]]

        # Calculate Propensity Scores
        L = recs[other_treatments]*originals[other_treatments]
        L += (1 -recs[other_treatments])*(1-originals[other_treatments])
        L = (L.apply(sum, axis=1) == N-2).astype(int)
        eta = q/(q + L*(1-exploration_prob))
        propensity_scores = eta/2 + L*originals[treat_1]*(1-eta)

        # Calculating Adjustment Formula
        inds = recs[treat_1] != recs[treat_2]
        diff_data = recs[inds]
        diff_data['propensity_scores'] = propensity_scores[inds]
        diff_ate = 0
        for control_vars, sub_data in diff_data.groupby(other_treatments + ['propensity_scores']):
            prop = sub_data.shape[0]/diff_data.shape[0]
            res = sub_data.groupby(treat_1).outcome.mean()
            diff_ate += (res.iloc[1] - res.iloc[0])*prop
        diff_treats[treat_1, treat_2] = diff_ate
            
pd.DataFrame(diff_treats.round(2))

,0,1,2,3,4,5,6,7,8,9
0,0.0,-0.37,-0.88,-1.14,-1.69,-1.52,-2.15,-2.52,-2.41,-3.46
1,0.0,0.00,-0.51,-0.85,-1.37,-1.35,-1.76,-1.90,-2.65,-3.12
2,0.0,0.00,0.00,-0.55,-0.81,-0.56,-1.33,-1.75,-1.95,-2.70
3,0.0,0.00,0.00,0.00,-0.53,-0.11,-0.93,-1.12,-1.61,-2.49
4,0.0,0.00,0.00,0.00,0.00,0.43,-0.40,-0.54,-0.96,-1.75
5,0.0,0.00,0.00,0.00,0.00,0.00,-0.83,-0.86,-1.08,-2.04
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.16,-0.64,-1.34
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-1.23
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.87
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In comparison with the expected results

In [16]:
pd.DataFrame(expected_diff_treats.round(2))

,0,1,2,3,4,5,6,7,8,9
0,0.0,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88,-3.24
1,0.0,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52,-2.88
2,0.0,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16,-2.52
3,0.0,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80,-2.16
4,0.0,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44,-1.80
5,0.0,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08,-1.44
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72,-1.08
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36,-0.72
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.36
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
